In [ ]:
%%html
<!-- definir quelques styles custom pour l'ensemble du notebook -->
<style>    
    @import url("css/custom_styles.css")
</style>

<center>
    <h1>
    Algorithmes d’apprentissage non supervisé<br>
    </h1>
    MovieLens - Système de recommendations de films par regroupement<br>
    <br>
    <b>Jean-Francois Gagnon</b><br>
    <b>Michèle de La Sablonnière</b><br>
    <br>
    420-A58<br>
    <br>
</center>

# Introduction
    
Nous avons choisi la base de données [MovieLens Small](https://tinyurl.com/bdhmcfht). Elle décrit les notations des utilisateurs de MovieLens; un service en ligne de recommandation de films. Elle a été préalablement traité afin d'extraire les informations intéressantes pour un système de suggestion par clusterisation et règles d'associations. Les résultats se trouvent dans 2 nouveaux fichiers. 

<br>**movies_pretraitement.csv**

Condensé des informations pertinente pour 1 film. Chaque ligne a ce format:

<div class="indentation">
<div class="fixblock"> movieId, imdbId, title, year, year_category, year_boxcox_std, rating_mode, genres, genres_tfidf_*</div>

|Attribut|<center>Description</center>|
|:-|:---|
|movieId| Identifiant du film dans cette base de données.|
|imdbId| Identifiant du film dans [Internet Movie Database](http://www.imdb.com).|
|title| Titre du film. Peut contenir l'année de parution.|
|year| Année de parution du film.|
|year_category| **year** catégorisé.|
|year_boxcox_std| **year** transformé par BoxCox et StandardScaler.|
|rating_mode| Nombre d'étoiles attribuées avec une granularité de $\frac{1}{2}$. |    
|genres| Genres. Liste de mots séparés par un 'pipe' (\|). |
|genres_tfidf_*| Genres encodé selon TF-IDF. Ici l'étoile représente toutes les modalités de **genres**. |
</div>

<br>**ratings_pretraitement.csv**

Ce fichier contient les métadonnées de chaque film. Chaque ligne a ce format:

**a valider**
<div class="indentation">
<div class="fixblock">userId, films*</div>

|Attribut|<center>Description</center>|
|:-|:---|
|userId| Identifiant de l'utilisateur.|
|films*| Identifiant du film. Ici l'étoile représente 1 colonne par film |
</div>

<br>

L'objectif de ce projet est de clusteriser.

In [ ]:
#
# imports utilitaires
#

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import pairwise_distances

#
# imports faisant partie de nos propres modules
#

import helpers as hlp
import helpers.Clustering as clstr
import helpers.jupyter as jup

# Lecteure des données

In [ ]:
#
# parametres configurant nos traitemens
#
configs = hlp.get_configs("config_overrides.json")

# imdbId doit etre garde en string (leading 0)
# voir la partie pretraitement pour comprendre pourquoi rating_mode et rating_median sont en string
movies_df = pd.read_csv("dataset/movies_pretraitement.csv", 
                        dtype={"imdbId": str})

print("Movies", movies_df.shape)
print("Head")
display(movies_df.head().round(2))
clstr.show_types(movies_df)
clstr.show_na(movies_df);

* 9460 individus, 26 variables
* aucune valeur manquante  
* *genres* est de type qualitatif. 
* *year* quantitatif.
* *rating_mode* sont qualitatif ordinal.

In [ ]:
quant_cols = ["year", "year_boxcox_std"]
quant_df = movies_df[quant_cols]

qual_cols = ["year_category", "genres"]
qual_df = movies_df[qual_cols]

tfidf_cols = movies_df.columns.str.contains("genres_tfidf_")
tfidf_cols = movies_df.columns[tfidf_cols]
genres_tfidf_df = movies_df[tfidf_cols]

## Distributions - variables quantitatives

In [ ]:
print("Stats générales - variables quantitatives")
display(quant_df.describe().T.round(2))
print()
print("Distributions - variables quantitatives")
clstr.show_distributions(quant_df, num_cols=2, figsize=(10, 2.5))

### *year*
Les films sont relativement récents: peak autour de l'an 2000. La base de données couvre un large spectre et la distribution est allongée: queue à gauche.
    
### *year_boxcox_std* 
Distrubution très près d'une gaussienne et multimodale. On voit bien l'effet des catégories. La majorités des votes sont entre [3, 4]

# Valeurs aberrantes - variables quantitatives

In [ ]:
clstr.show_outliers_iqr(quant_df, 
                        eta=1.5,
                        boxlists=[["year"], ["year_boxcox_std"]],
                        figsize=(7, 2.5));

Dire qu'on prend **year_boxcox_std**

# Distributions - variables qualitatives

Dire que tf-idf n'a pas a etre afficher comme distribution.

In [ ]:
year_category_ohe_df = qual_df.year_category.str.get_dummies()
year_category_count = year_category_ohe_df.sum(axis=0)

genres_ohe_df = qual_df.genres.str.get_dummies(sep=configs.dataset.genre_splitter)
genres_count = genres_ohe_df.sum(axis=0)
genres_count.sort_values(ascending=False, inplace=True)


plt.figure(figsize=(8, 3))

plt.subplot(121)
plt.bar(year_category_count.index, year_category_count)
plt.title("year_category")

plt.subplot(122)
plt.bar(genres_count.index, genres_count)
plt.tick_params(axis="x", labelrotation=90)
plt.title("genres")
             
plt.tight_layout()
plt.show()

* Forte proportion dans Drama et Comedy

# Corrélation et reduction de dimensions

TODO

# Clustering

Les variables retenues sont donc:
    
* genres
* year_category
* rating_mode

On va aussi essayer tf-idf

In [ ]:
erreur volontaire ici

In [ ]:
# tsvd_sample = TruncatedSVD(n_components=31)
# tsvd_sample_df = tsvd_sample.fit_transform(acm_dummies_sample_df)

# ss = np.cumsum(tsvd_sample.explained_variance_ratio_)

# plt.plot(range(1, ss.shape[0] + 1), ss, marker=".")
# plt.grid(True)
# plt.show

# plt.scatter(tsvd_sample_df[:, 1], tsvd_sample_df[:, 2], s=0.5)
# plt.show()

In [ ]:
class MCA_from_dummies(Base):
    def __init__(self, n_components=None, row_labels=None, var_labels=None, stats=True):
        Base.__init__(self, n_components, row_labels, None, stats)
        self.var_labels = var_labels
        
    def prefit(self, X, X_dummies):
        # Set columns prefixes
        self.n_vars_ = X.shape[1]
        self.prefixes_ = self.prefixes_ = [str(x) + "_" for x in self.var_labels]
        self.col_labels_short_temp_ = [x.split("_")[-1] for x in X_dummies.columns]
        self.col_labels_temp_ = X_dummies.columns
                
        # Dummy variables creation
        self.n_categories_ = X_dummies.shape[1]
    
    def fit(self, X_dummies, y=None):
        # Fit a Factorial Analysis to the dummy variables table
        self.r_ = np.sum(X_dummies, axis=1).reshape(-1, 1)
        Base.fit(self, X_dummies, y=None)
        
        # Adjustment of the number of components
        n_eigen = self.n_categories_ - self.n_vars_
        if (self.n_components_ > n_eigen):
            self.n_components_ = n_eigen
            self.eig_ = self.eig_[:, :self.n_components_]
            self.row_coord_ = self.row_coord_[:, :self.n_components_]
            self.col_coord_ = self.col_coord_[:, :self.n_components_]
            if self.stats:
                self.row_contrib_ = self.row_contrib_[:, :self.n_components_]
                self.col_contrib_ = self.col_contrib_[:, :self.n_components_]
                self.row_cos2_ = self.row_cos2_[:, :self.n_components_]
                self.col_cos2_ = self.col_cos2_[:, :self.n_components_]

        # Set col_labels_short_
        self.col_labels_short_ = self.col_labels_short_temp_
        
        # Set col_labels_
        self.col_labels_ = self.col_labels_temp_        
        
        self.model_ = "mca"
        
        return self
                        
def acm_init(data, n_components, data_dummies=None):
    if data_dummies is None:
        acm = MCA(n_components=n_components,
                  row_labels=data.index,
                  var_labels=data.columns)
        acm.fit(data.to_numpy())
    else:
        acm = MCA_from_dummies(n_components=n_components,
                               row_labels=data.index,
                               var_labels=data.columns)
        
        acm.prefit(data, data_dummies)        
        acm.fit(data_dummies.to_numpy())
    
    return acm

def acm_analysis(data, data_dummies=None, figsize=(4, 2.5)):
    """
    Le threshold est ~60% sur cumul var. expliquee
    """
    acm = acm_init(data, None, data_dummies=data_dummies)

    threshold = 1 / acm.n_vars_
    eig_vals = acm.eig_[0]
    eig_th = eig_vals[eig_vals > threshold]

    print("Valeurs propres:")
    print(acm.eig_[0].round(4))
    print()
    print(f"Valeurs propres > {round(threshold, 4)} (1 / p):")
    print(eig_th.round(4))
    print()
    print("Variance expliquee %:")
    print(acm.eig_[1].round(1))
    print()
    print("Variance expliquee cumul. %:")
    print(acm.eig_[2].round(1))
    print()

    num_eigval = len(acm.eig_[0])

    plt.figure(figsize=figsize)
    plt.plot(range(1, num_eigval + 1), acm.eig_[0], marker=".")
    plt.grid(True)
    plt.xlabel("# axe factoriel")
    plt.ylabel("Valeur propre")
    plt.show()

acm_analysis(data=acm_sample_df, data_dummies=acm_dummies_sample_df)

In [ ]:
acm_ = acm_init(data=acm_sample_df, data_dummies=acm_dummies_sample_df, n_components=21)

plt.scatter(acm_.row_coord_[:, 0], acm_.row_coord_[:, 1], s=0.5)
plt.show()

In [ ]:
clstr.cah_analysis(acm_.row_coord_)

In [ ]:
cah_movies = clstr.cah_init(acm_.row_coord_, 5)

print("CAH clusters")
clstr.clusters_analysis(acm_.row_coord_, cah_movies.labels_)
clstr.show_clusters(acm_.row_coord_[:, [0, 1]], 
                    acm_.row_labels_,
                    cah_movies.labels_,
                    text_alpha=0,
                    marker_size=2.5)

In [ ]:
clstr.dbscan_eps_analysis(acm_.row_coord_)

In [ ]:
eps_, min_samples_ = clstr.dbscan_parameters_analysis(acm_.row_coord_,
                                                      np.arange(0.4, 1.4, 0.1),
                                                      range(3, 20))
dbscan_movies = clstr.dbscan_init(acm_.row_coord_, eps_, min_samples_)

print()
print("DBSCAN clusters")
clstr.clusters_analysis(acm_.row_coord_, dbscan_movies.labels_)

In [ ]:
clstr.show_clusters(acm_.row_coord_[:, [0, 1]], 
                    acm_.row_labels_,
                    dbscan_movies.labels_,
                    text_alpha=0,
                    marker_size=2.5)

In [ ]:
from kmodes.kmodes import KModes

kmodes_movies = KModes(n_clusters=4, init='Huang', n_init=5, verbose=1, n_jobs=-1)
kmodes_movies.fit(acm_.row_coord_)

print("KModes clusters")
clstr.clusters_analysis(acm_.row_coord_, kmodes_movies.labels_)

clstr.show_clusters(acm_.row_coord_[:, [0, 1]], 
                    acm_.row_labels_,
                    kmodes_movies.labels_,
                    text_alpha=0,
                    marker_size=2.5)